In [1]:
from pytorch_unet import *
from torch import nn
import pandas as pd
from timerit import Timerit
from IPython.display import display, HTML

In [2]:
print(torch.__version__)

1.9.0+cu102


In [3]:
model = UNet_rect_kernels(128,2).eval()

x = torch.randn((2,1,128,128))
pred = model(x)

In [4]:
device_name = " "
if torch.cuda.is_available():
    device_name = torch.cuda.get_device_name(torch.cuda.current_device())
else:
    device_name = 'cpu'
    
print('Using device:', device_name)

device = " "
if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'

Using device: GeForce RTX 2060 SUPER


In [6]:
metric_labels = [
    "Frames",
    "# Samples",
    "First PT (s)",
    "Avg PT (s)",
    "Min PT (s)",
    "Max PT (s)",
    "Std. Dev. (s)",
]
    
print('Using device:', device_name)

results_df = pd.DataFrame(columns = metric_labels)
frames = [2, 10, 20, 30, 40, 50]
for l in range(len(frames)):

    model = UNet(128,2)
    if device == 'cuda':
        model = model.to(torch.device('cuda'))
    t = Timerit(num = 100, verbose=0)
    
    model = model.eval()
    
    for timer in t:
        x = torch.randn((frames[l],1,128,128))
        with timer:
            x = x.to(device)
            pred = model(x)
    
    mf_times = t.times
    
    results_df.loc[l] = [
        frames[l],
        100,
        mf_times[0],
        int((np.mean(mf_times)*100000)) / 100000,
        np.min(mf_times),
        np.max(mf_times),
        int((np.std(mf_times)*100000)) / 100000
    ]

print('Pytorch Multi-Frame Batch Prediction Timings (rect kernels)')
print('"PT" = Prediction Time')
display(results_df)

Using device: GeForce RTX 2060 SUPER
Pytorch Multi-Frame Batch Prediction Timings (rect kernels)
"PT" = Prediction Time


,Frames,# Samples,First PT (s),Avg PT (s),Min PT (s),Max PT (s),Std. Dev. (s)
0,2.0,100.0,0.003001,0.00260,0.002556,0.003001,0.00007
1,10.0,100.0,0.003007,0.00268,0.002656,0.003007,0.00003
2,20.0,100.0,0.003065,0.00288,0.002732,0.003444,0.00017
3,30.0,100.0,0.003396,0.00296,0.002919,0.003396,0.00005
4,40.0,100.0,0.003377,0.00308,0.002999,0.003492,0.00010
5,50.0,100.0,0.003455,0.00334,0.003125,0.003625,0.00014
